In [1]:
import pandas as pd
import numpy as np
import numpy.random as nr
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
import lightgbm as lgb
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [3]:
#importing files
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('SampleSubmission.csv')

In [4]:
train.head()

,Item_ID,Store_ID,Item_Store_ID,Item_Weight,Item_Sugar_Content,Item_Visibility,Item_Type,Item_Price,Store_Start_Year,Store_Size,Store_Location_Type,Store_Type,Item_Store_Returns
0,DRA12,BABATUNJI010,DRA12_BABATUNJI010,11.6,Low Sugar,0.068535,Soft Drinks,357.54,2005,NaN,Cluster 3,Grocery Store,709.08
1,DRA12,BABATUNJI013,DRA12_BABATUNJI013,11.6,Low Sugar,0.040912,Soft Drinks,355.79,1994,High,Cluster 3,Supermarket Type1,6381.69
2,DRA12,BABATUNJI017,DRA12_BABATUNJI017,11.6,Low Sugar,0.041178,Soft Drinks,350.79,2014,NaN,Cluster 2,Supermarket Type1,6381.69
3,DRA12,BABATUNJI018,DRA12_BABATUNJI018,11.6,Low Sugar,0.041113,Soft Drinks,355.04,2016,Medium,Cluster 3,Supermarket Type2,2127.23
4,DRA12,BABATUNJI035,DRA12_BABATUNJI035,11.6,Ultra Low Sugar,0.000000,Soft Drinks,354.79,2011,Small,Cluster 2,Supermarket Type1,2481.77


In [5]:
train.shape, test.shape

((4990, 13), (3532, 12))

In [6]:
train.isna().sum()

Item_ID                   0
Store_ID                  0
Item_Store_ID             0
Item_Weight             802
Item_Sugar_Content        0
Item_Visibility           0
Item_Type                 0
Item_Price                0
Store_Start_Year          0
Store_Size             1450
Store_Location_Type       0
Store_Type                0
Item_Store_Returns        0
dtype: int64

In [7]:
train.dtypes

Item_ID                 object
Store_ID                object
Item_Store_ID           object
Item_Weight            float64
Item_Sugar_Content      object
Item_Visibility        float64
Item_Type               object
Item_Price             float64
Store_Start_Year         int64
Store_Size              object
Store_Location_Type     object
Store_Type              object
Item_Store_Returns     float64
dtype: object

In [ ]:
#filling missing values
train['Item_Weight'] = train['Item_Weight'].fillna(train['Item_Weight'].mean())
train['Supermarket_Size'].fillna(train['Supermarket_Size'].mode()[0],inplace=True)
test['Item_Weight'] = test['Item_Weight'].fillna(test['Item_Weight'].mean())
test['Supermarket_Size'].fillna(test['Supermarket_Size'].mode()[0],inplace=True)

#dropping unwanted features
train = train.drop(['Product_Identifier', 'Product_Supermarket_Identifier'], axis=1)
test = test.drop(['Product_Identifier', 'Product_Supermarket_Identifier'], axis=1)

#converting the year to age of supermarket
train['Supermarket_Opening_Year'] = 2019 - train['Supermarket_Opening_Year']
test['Supermarket_Opening_Year'] = 2019 - test['Supermarket_Opening_Year']

#creating labels
labels = np.array(train['Product_Supermarket_Sales'])
train = train.drop('Product_Supermarket_Sales', axis=1)